In [67]:
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import shapely.geometry as sgeom
import cartopy.io.img_tiles as cimgt
import plotly.express as px
import plotly.graph_objects as go





In [68]:
pd.set_option("display.max_rows", None) 
pd.set_option("display.max_columns", None)

In [69]:
#df = pd.read_json('/Users/barry/CodeAcademy/Sea_watch/sample_traces/2022_09_407637.json')
df = pd.read_json('sample_traces/2021_01_346401.json')

In [ ]:
import os
import types

import cartopy.io.img_tiles as img_tiles
import requests
import PIL

class CachedTiler(object):
    def __init__(self, tiler):
        self.tiler = tiler

    def __getattr__(self, name):
        attr = getattr(self.tiler, name, None)
        if isinstance(attr, types.MethodType):
            attr = types.MethodType(attr.__func__, self)
        return attr

    def get_image(self, tile):
        tileset_name = '{}'.format(self.tiler.__class__.__name__.lower())
        cache_dir = os.path.expanduser(os.path.join('~/', 'image_tiles', tileset_name))
        if not os.path.exists(cache_dir):
            os.makedirs(cache_dir)
        tile_fname = os.path.join(cache_dir, '_'.join(str(v) for v in tile) + '.png')
        if not os.path.exists(tile_fname):
            response = requests.get(self._image_url(tile),
                                    stream=True)

            with open(tile_fname, "wb") as fh:
                for chunk in response:
                    fh.write(chunk)
        with open(tile_fname, 'rb') as fh:
            img = PIL.Image.open(fh)
            img = img.convert(self.desired_tile_form)     
        return img, self.tileextent(tile), 'lower'


In [ ]:
def route_maker(df):

    if 'icao' in df.columns:
        icao = df.icao.unique()[0]
    elif 'hex' in df.columns:
        icao = df.hex.unique()[0]
    time_stamp = df.timestamp.unique()[0]
    timeline = []
    latitude = []
    longitude = []
    altitude = []

    for val in df.trace:
        timeline.append(val[0])
        latitude.append(val[1])
        longitude.append(val[2])
        altitude.append(val[3])
   
    print(f'plane: {icao}\ntake_off: {time_stamp}')
    flight_info = zip(timeline, latitude, longitude, altitude)

    # Initialize our Google Maps tiles
    actual_tiler = cimgt.GoogleTiles()
    imagery = CachedTiler(actual_tiler)


    fig = plt.figure(figsize=(10,10), facecolor='white')
    ax = fig.add_subplot(projection=ccrs.LambertConformal())
    plt.title(f'{icao} departing: {time_stamp}')

    # Set the boundaries around the map
    ax.set_extent([ 
        max(longitude) +2, min(longitude) - 2,
        min(latitude) -2 , max(latitude) +2
    ], crs=ccrs.Geodetic())

    # Draw the background
    ax.add_image(imagery, 9)

    # Draw the path of the fligt
    track = sgeom.LineString(zip(longitude,latitude))
    ax.add_geometries([track],
                      ccrs.PlateCarree(),
                      facecolor='none',
                      edgecolor='red',
                      linewidth=2)    
    print(len(latitude))
    print(len(longitude))
    print(len(altitude))
route_maker(df)



route_maker3 takes a df and hten creates an interative plotly map showing altitude as well as location

In [70]:
def route_maker3(json_name):
    df = pd.read_json(f'sample_traces/{json_name}.json')
    if 'icao' in df.columns:
        icao = df.icao.unique()[0]
    elif 'hex' in df.columns:
        icao = df.hex.unique()[0]
    time_stamp = df.timestamp.unique()[0]
    print(type(time_stamp))
    timeline = []
    latitude = []
    longitude = []
    altitude = []

    for val in df.trace:
        timeline.append(val[0])
        latitude.append(val[1])
        longitude.append(val[2])
        altitude.append(val[3])
    print(f'plane: {icao}\ntake_off: {time_stamp}')
    #flight_info = zip(timeline, latitude, longitude, altitude)

    df2 = pd.DataFrame({
    'Latitude': latitude,
    'Longitude': longitude,
    'Altitude': altitude,
    'Time': timeline
    })
    df2['index'] = df2.index
    #I need the altitude to be an integer no Nulls or strings permitted
    df2.loc[df2.Altitude == 'ground', 'Altitude'] = 0
    df2.Altitude = df2.Altitude.fillna(0)
    df2['Altitude'] = df2['Altitude'].astype('int')
    
    df2.Time = pd.to_timedelta(df2.Time) 
    df2.Time = df2.Time + time_stamp


    # Plot flight path with altitude on a Mapbox map
    fig = px.scatter_mapbox(df2, 
                            lon='Longitude', 
                            lat='Latitude', 
                            color='Altitude', 
                            hover_data=['index', 'Longitude', 'Latitude','Time'], 
                         title=f'Flight {icao} take-off: {time_stamp}')
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_traces(mode='markers+lines', line=dict(color='blue', width=2))
    fig.update_layout(height=800, width=1200) 

    fig.show()

route_maker3('2021_01_346401')

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
plane: 346401
take_off: 2021-01-01 23:49:40.979000064
